#### Setup

In [1]:
import sys
sys.path.append('../input/einops')
sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')
sys.path.append('../input/coatmodeltrain')
#from efficientnet_pytorch import EfficientNet
!pip install ../input/einops-030/einops-0.3.0-py2.py3-none-any.whl
# !pip install --upgrade efficientnet-pytorch
#import efficientnet_pytorch

#!python3 ../input/coatmodeltrain/CoatModel.py
#from coatmodeltrain import *
#!python3 ../input/coatmodeltrain/EfficientModelTrain.py

Processing /kaggle/input/einops-030/einops-0.3.0-py2.py3-none-any.whl


In [2]:
import albumentations as A


In [3]:
sys.path.append('../input/coatmodeltrain/')


In [ ]:
from CoatModel import *

In [4]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs, image_size):
    best_acc = 0.0
    best_loss = 10.0
    for epoch in range(num_epochs):
        model.cuda()       
        for phase in ['train', 'val']:
            if phase == 'train': model.train()
            else: model.eval()
            epoch_loss = 0.0
            epoch_acc = 0
            
            
            dataloader = dataloaders_dict[phase]
            for item in tqdm(dataloader, leave=False):
                images = item[0].cuda().float()
                classes = item[1].cuda().long()
                optimizer.zero_grad()                
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(images)
                    loss = criterion(output, classes)
                    _, preds = torch.max(output, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    epoch_loss += loss.item() * len(output)
                    epoch_acc += torch.sum(preds == classes.data)   
                del images
                del classes
                gc.collect()
            data_size = len(dataloader.dataset)
            epoch_loss = epoch_loss / data_size
            epoch_acc = epoch_acc.double() / data_size
            print(f'Epoch {epoch + 1}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')    
#         if epoch_acc > best_acc:
#             traced = torch.jit.trace(model.cpu(), torch.rand(1, 3, image_size, image_size))
#             traced.save('model.pth')
#             best_acc = epoch_acc
#             del traced
#             gc.collect()
        if epoch_loss < best_loss:
            traced = torch.jit.trace(model.cpu(), torch.rand(1, 3, image_size, image_size))
            traced.save('model_Best_loss.pth')
            best_loss = epoch_loss
            del traced
            gc.collect()
 


In [10]:
debug = False
generate_new = False
train_df = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv").head(1000)
test_df = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
dirs = ["../input/mayo-clinic-strip-ai/train/", "../input/mayo-clinic-strip-ai/test/"]

In [12]:
from PIL import Image

class ImgDataset(Dataset):
    def __init__(self, df, stage):
        self.df = df 
        self.train = 'label' in df.columns    
        
        if stage == 'train':
            self.transforms = A.Compose([
                A.augmentations.Rotate(limit=90, p=0.5),
                A.augmentations.HorizontalFlip(p=0.5),
                A.augmentations.VerticalFlip(p=0.5),
                A.augmentations.transforms.ColorJitter(p=0.5),
            
                A.OneOf([
                    A.HueSaturationValue(10, 15, 10),
                    A.RandomBrightnessContrast(),            
                ], p=0.5),                
                A.Normalize(),
            ])
        else:
            self.transforms = A.Compose([
                A.Normalize(),
            ])
        
        
        
    def __len__(self): return len(self.df)    
    def __getitem__(self, index):
        if(generate_new): paths = ["./test/", "./train/"]
        else: paths = ["../input/jpg-images-strip-ai/test/", "../input/mayoclinicai-224456600-resized-training-images/resized_training_dataset/img_1024_folder/"]
        
        if paths == "../input/jpg-images-strip-ai/test/":
            image = cv2.imread(paths[self.train] + self.df.iloc[index].image_id + ".jpg")
        else:
            image = cv2.imread(paths[self.train] + self.df.iloc[index].image_id + ".jpg")
            
        
        #image = cv2.resize(image, (512, 512))#.transpose(2, 0, 1)
        #image = cv2.resize(tifffile.imread(paths[self.train] + self.df.iloc[index].image_id + ".jpg"), (512, 512))
        
        img = self.transforms(image=cv2.resize(image, (512, 512)))["image"]
        
        if len(img.shape) == 5:
            img_squeeze = img.squeeze().transpose(1, 2, 0)
        image = img_squeeze.transpose(2, 0, 1)
        label = None
        if(self.train): label = {"CE" : 0, "LAA": 1}[self.df.iloc[index].label]
        del img
        del img_squeeze
        gc.collect()  
        return image, label

#### Architecture Definition & Running the training

In [ ]:
num_blocks = [2, 6, 12, 28, 6] #[2, 2, 6, 8, 2]
channels = [64, 64, 128, 256, 512]
#model = torch.jit.load('../input/coatmodeltrain/model.pth')
model = CoAtNet((512, 512), 3, num_blocks, channels, num_classes=2)
train, val = train_test_split(train_df, test_size=0.2, random_state=42, stratify = train_df.label)
batch_size = 1
train_loader = DataLoader(ImgDataset(train, 'train'), batch_size=4, shuffle=False, num_workers=2)
val_loader = DataLoader(ImgDataset(val, 'val'), batch_size=batch_size, shuffle=False, num_workers=1)
dataloaders_dict = {"train": train_loader, "val": val_loader}
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=8e-5)
train_model(model, dataloaders_dict, criterion, optimizer, 30, 512)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
train_model(model, dataloaders_dict, criterion, optimizer, 30, 512)

  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/30 | train | Loss: 0.9023 | Acc: 0.6434


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/30 |  val  | Loss: 0.5886 | Acc: 0.7285


  0%|          | 0/151 [00:00<?, ?it/s]

...

Epoch 8/30 |  val  | Loss: 0.5871 | Acc: 0.7285


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 9/30 | train | Loss: 0.6091 | Acc: 0.7148


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 9/30 |  val  | Loss: 0.5488 | Acc: 0.7417


In [1]:
ls

__notebook_source__.ipynb


#### EfficientNet B4 - Training

In [4]:
#from EfficientModelTrain import *
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [8]:
import os
import gc
import cv2
import copy
import time
import torch
import random
import string
import joblib
import tifffile
import numpy as np 
import pandas as pd 
from torch import nn
import seaborn as sns
#import efficientnet_pytorch
from torchvision import models
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import warnings; warnings.filterwarnings("ignore")

**Setups**

In [5]:
!mkdir /root/.cache/torch
!mkdir /root/.cache/torch/hub
!mkdir /root/.cache/torch/hub/checkpoints
!cp -r ../input/torchhub-efficientnet-b7/nvidia_efficientnet-b7_210412.pth /root/.cache/torch/hub/checkpoints/

cp: cannot stat '../input/torchhub-efficientnet-b7/nvidia_efficientnet-b7_210412.pth': No such file or directory


In [6]:
    
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs, img_size):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.cuda()
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_acc = 0
            
            dataloader = dataloaders_dict[phase]
            for item in tqdm(dataloader, leave=False):
                images = item[0].cuda().float()
                classes = item[1].cuda().long()

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(images)
                    loss = criterion(output, classes)
                    _, preds = torch.max(output, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * len(output)
                    epoch_acc += torch.sum(preds == classes.data)
                    

            data_size = len(dataloader.dataset)
            epoch_loss = epoch_loss / data_size
            epoch_acc = epoch_acc.double() / data_size

            print(f'Epoch {epoch + 1}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')
        
        if epoch_acc > best_acc:
            traced = torch.jit.trace(model.cpu(), torch.rand(1, 3, img_size, img_size))
            traced.save('efficientnet_model.pth')
            best_acc = epoch_acc

#### Running the training

In [ ]:
model = efficientnet_pytorch.EfficientNet.from_name("efficientnet-b7")
checkpoint = torch.load('../input/efficientnet-pytorch/efficientnet-b7-dcc49843.pth')
model.load_state_dict(checkpoint)

train, val = train_test_split(train_df, test_size=0.2, random_state=42, stratify = train_df.label)
batch_size = 1
train_loader = DataLoader(ImgDataset(train, 'train'), batch_size=4, shuffle=False, num_workers=1)
val_loader = DataLoader(ImgDataset(val, 'val'), batch_size=batch_size, shuffle=False, num_workers=1)
dataloaders_dict = {"train": train_loader, "val": val_loader}
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
train_model(model, dataloaders_dict, criterion, optimizer, 2,512)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
train_model(model, dataloaders_dict, criterion, optimizer, 10, 512)

  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/2 | train | Loss: 1.5641 | Acc: 0.6368


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/2 |  val  | Loss: 0.7114 | Acc: 0.6689


  0%|          | 0/151 [00:00<?, ?it/s]

...

  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 4/10 | train | Loss: 0.4883 | Acc: 0.7944


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 4/10 |  val  | Loss: 0.5969 | Acc: 0.7219


  0%|          | 0/151 [00:00<?, ?it/s]

### Efficientnet Inference

In [19]:
import os
import gc
import cv2
import copy
import time
import random
import torch
import string
import joblib
import tifffile
import numpy as np 
import pandas as pd 
from torch import nn
import seaborn as sns
import efficientnet_pytorch
from tqdm.notebook import tqdm
from torchvision import models
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import warnings; warnings.filterwarnings("ignore")

gc.enable()
debug = False
generate_new = True
test_df = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
dirs = ["../input/mayo-clinic-strip-ai/train/", "../input/mayo-clinic-strip-ai/test/"]
test_df

try:
    os.mkdir("../test/")
except:
    pass
for i in tqdm(range(test_df.shape[0])):
    img_id = test_df.iloc[i].image_id
    try:
        sz = os.path.getsize(dirs[1] + img_id + ".tif")
    except:
        sz = 1000000000
    if(sz > 8e8):
        img = np.zeros((768,768,3), np.uint8)
    else:
        try:
            img = cv2.resize(tifffile.imread(dirs[1] + img_id + ".tif"), (768, 768))
        except:
            img = np.zeros((768,768,3), np.uint8)
    cv2.imwrite(f"../test/{img_id}.jpg", img)
    del img
    gc.collect()
    
    
class ImgDataset(Dataset):
    def __init__(self, df):
        self.df = df 
        self.train = 'label' in df.columns
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        if(generate_new):
            paths = ["../test/", "../train/"]
        else:
            paths = ["../input/jpg-images-strip-ai/test/", "../input/jpg-images-strip-ai/train/"]
        try:
            image = cv2.imread(paths[self.train] + self.df.iloc[index].image_id + ".jpg")
        except:
            image = np.zeros((768,768,3), np.uint8)
        label = 0
        try:
            if len(image.shape) == 5:
                image = image.squeeze().transpose(1, 2, 0)
            image = cv2.resize(image, (768, 768)).transpose(2, 0, 1)
        except:
            image = np.zeros((3, 768, 768))
        if(self.train):
            label = {"CE" : 0, "LAA": 1}[self.df.iloc[index].label]
        patient_id = self.df.iloc[index].patient_id
        return image, label, patient_id
def predict(model, dataloader):
    model.cuda()
    model.eval()
    dataloader = dataloader
    outputs = []
    s = nn.Softmax(dim=1)
    ids = []
    for item in tqdm(dataloader, leave=False):
        patient_id = item[2][0]
        try:
            images = item[0].cuda().float()
            ids.append(patient_id)
            output = model(images)
            outputs.append(s(output.cpu()[:,:2])[0].detach().numpy())
        except:
            ids.append(patient_id)
            outputs.append(s(torch.tensor([[1, 1]]).float())[0].detach().numpy())
    return np.array(outputs), ids


model = torch.jit.load('efficientnet_model.pth')
batch_size = 1
test_loader = DataLoader(
    ImgDataset(test_df), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=1
)
anss, ids = predict(model, test_loader)

prob = pd.DataFrame({"CE" : anss[:,0], "LAA" : anss[:,1], "id" : ids}).groupby("id").mean()
submission = pd.read_csv("../input/mayo-clinic-strip-ai/sample_submission.csv")
submission.CE = prob.CE.to_list()
submission.LAA = prob.LAA.to_list()
submission.to_csv("submission_efficientnet.csv", index = False)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

# Ensemble

Now that we got both our models ready, we can simply combine the predictions of both and use them as an ensemble!
We do this by running the predictions and averaging them. This yields a more powerful model.

In [20]:
submission_coatnet = pd.read_csv("submission_coatnet.csv")
submission_efficientnet = pd.read_csv("submission_efficientnet.csv")
sub_df = pd.read_csv('../input/mayo-clinic-strip-ai/sample_submission.csv')

sub_df['CE'] = (submission_efficientnet['CE'].values + submission_coatnet['CE'].values) / 2.0
sub_df['LAA'] = (submission_efficientnet['LAA'].values + submission_coatnet['LAA'].values) / 2.0


sub_df.to_csv('submission.csv', index = False)

In [21]:
from IPython.display import FileLink

#%cd ../../..
!pwd
#!cp -r ./00003570.model.pth .
import os
#os.chdir(r'/kaggle/working')
os.chdir(r'/kaggle/working/')
FileLink(r'model.pth')

/kaggle/working


/kaggle/working/model.pth